In [1]:
import pandas as pd

/var/folders/tk/lg_kp6zd7k3261vdk48m7jch0000gn/T/ipykernel_3319/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
case_df = pd.read_csv('heme_diff_complete.csv')
case_df

,Accession Number,WHO Chapter,General Dx,Dx_Cat,Age,text_data_clindx,text_data_final,Blast,Promyelocyte,Myelocyte,Metamyelocyte,Neutrophils/Band,Monocyte,Eosinophil,Basophil,Erythroid Precursor,Lymphocyte,Plasma Cell,Diff_Sum,M:E Ratio
0,H15-1568,Acute myeloid leukemia and related precursor n...,AML,Acute Myeloid Leukemia/Acute Lymphoblastic Leu...,89,\n\n\n\n\n\n\n\n\n89 M with newly diagnosed AM...,"\n\n\n\n\n\n\n\n1. Bone marrow, right posterio...",75.0,NaN,NaN,NaN,1.0,4.0,NaN,NaN,8.0,12.0,NaN,100,NaN
1,H15-1801,Acute myeloid leukemia and related precursor n...,AML,Acute Myeloid Leukemia/Acute Lymphoblastic Leu...,18,\n\n\n\n\n\n\n\n\n\n18 F with MDS (RAEB-2 with...,"\n\n\n\n\n\n1\n. Bone marrow, right posterior ...",24.0,1.0,NaN,NaN,7.0,1.0,NaN,NaN,27.0,40.0,NaN,100,0.3
2,H15-2010,Acute myeloid leukemia and related precursor n...,AML,Acute Myeloid Leukemia/Acute Lymphoblastic Leu...,44,\n\n\n\n\n\n\n\n\n\n\nAML. Pancytopenic.\n\n\n\n,"\n\n\n\n\n\n\n\n\n1. \nBone marrow, RPIC; aspi...",80.0,NaN,1.0,NaN,1.0,8.0,NaN,NaN,NaN,9.0,NaN,99,0.0
3,H15-2554,Acute myeloid leukemia and related precursor n...,AML,Acute Myeloid Leukemia/Acute Lymphoblastic Leu...,49,\n\n\n\n\n\n\n\n\n\nHigh-grade MDS.\n\n\n\n,"\n\n\n\n\n\n1. Bone marrow, right posterior il...",35.0,NaN,NaN,NaN,5.0,4.0,NaN,NaN,20.0,35.0,NaN,99,0.5
4,H15-2805,Acute myeloid leukemia and related precursor n...,AML,Acute Myeloid Leukemia/Acute Lymphoblastic Leu...,80,\n\n\n\n\n\n\n\n\n\nAML. Screen for protocol 1...,"\n\n\n\n\n\n\n\n1. Bone marrow, right posterio...",27.0,4.0,15.0,5.0,8.0,1.0,1.0,NaN,25.0,14.0,NaN,100,1.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1213,H22-9370,Mature B-cell neoplasm,Plasma cell myeloma,Plasma Cell Myeloma,84,\n\n\n\n\n\nClinical Diagnosis: CLL\nClinical ...,"\n\n\n\n\n1-3. Bone marrow, right posterior il...",1.0,NaN,6.0,3.0,19.0,2.0,3.0,NaN,20.0,43.0,3.0,100,1.6
1214,H22-9381,Mature B-cell neoplasm,Plasma cell myeloma,Plasma Cell Myeloma,70,\n\n\n\n\n\nClinical Diagnosis: Multiple Myelo...,"\n\n\n\n\n1-3. Bone marrow, right posterior il...",2.0,1.0,5.0,5.0,28.0,NaN,4.0,NaN,28.0,14.0,13.0,100,1.5
1215,H22-9411,Mature B-cell neoplasm,Plasma cell myeloma,Plasma Cell Myeloma,68,\n\n\n\n\n\nClinical Diagnosis: Amyloid\nClini...,"\n\n\n\n\n1-3. Bone marrow, right posterior il...",1.0,1.0,7.0,5.0,39.0,7.0,2.0,NaN,25.0,10.0,3.0,100,2.4
1216,H22-9425,Mature B-cell neoplasm,Plasma cell myeloma,Plasma Cell Myeloma,66,\n\n\n\n\n\nClinical Diagnosis: Multiple Myelo...,"\n\n\n\n\n1-3. Bone marrow, left posterior ili...",1.0,1.0,8.0,4.0,40.0,2.0,4.0,NaN,23.0,9.0,8.0,100,2.6


In [3]:
def extract_year(case_id):
    year_prefix = '20' if case_id[1:3] <= '24' else '19'
    return int(year_prefix + case_id[1:3])

In [4]:
case_df['Year'] = case_df['Accession Number'].apply(extract_year)

In [6]:
cbc_df = pd.read_excel('/Users/hasan/Library/Mobile Documents/com~apple~CloudDocs/Projects/Heme Report OpenAI/CBC_H18-H23_DataLine Results - PTH29594.xlsx', sheet_name='Results')

In [7]:
cbc_df['Result_with_Unit'] = cbc_df['Result Value'].astype(str) + " " + cbc_df['Unit/Measure'].fillna('')

In [8]:
cbc_df['Pathology Procedure Date'] = pd.to_datetime(cbc_df['Pathology Procedure Date'])
most_recent_dates = cbc_df.groupby('Pathology Report')['Pathology Procedure Date'].max().reset_index()
filtered_cbc_df = cbc_df.merge(most_recent_dates, on=['Pathology Report', 'Pathology Procedure Date'])
pivoted_df = filtered_cbc_df.pivot_table(index='Pathology Report', 
                                               columns='Subtest Name', 
                                               values='Result Value',
                                               #values = 'Result_with_Unit', # Issues with Units so i'm just taking value
                                               aggfunc='first').reset_index()
pivoted_df = pivoted_df.fillna('')

In [9]:
case_df = case_df.merge(pivoted_df, how='left', left_on='Accession Number', right_on='Pathology Report')

In [10]:
case_df_cbc = case_df[case_df['Pathology Report'].notna()]
case_df_cbc

,Accession Number,WHO Chapter,General Dx,Dx_Cat,Age,text_data_clindx,text_data_final,Blast_x,Promyelocyte,Myelocyte,...,Mean Corpuscular Hemoglobin (MCH),Mean Corpuscular Hemoglobin Conc (MCHC),Mean Corpuscular Volume (MCV),Mono,Neutrophil,Nucleated RBC,Platelets.,RBC,Red Blood Cell Distribution Width (RDW),WBC
223,H19-1037,Acute myeloid leukemia and related precursor n...,AML,Acute Myeloid Leukemia/Acute Lymphoblastic Leu...,37,AML. Cytopenias.,"1-2. Bone marrow, right posterior iliac crest,...",73.0,2.0,2.0,...,32.0,33.3,96,4.0,2.0,0,22,2.22,16.8,33.8
224,H19-1359,Acute myeloid leukemia and related precursor n...,AML,Acute Myeloid Leukemia/Acute Lymphoblastic Leu...,50,AML. MSK Clinic Appointment: (02/18/2019).,"1. Bone marrow aspirates (C18-5669, 05/02/18, ...",17.0,NaN,9.0,...,30.2,32.4,93,1.0,3.0,0,3,2.81,16.0,5.0
225,H19-1422,Acute myeloid leukemia and related precursor n...,AML,Acute Myeloid Leukemia/Acute Lymphoblastic Leu...,32,AML. MDS now new diagnosis of AML (20% circula...,"1-3. Bone marrow, right posterior iliac crest;...",22.0,3.0,12.0,...,32.2,32.4,99,5.0,47.0,1,72,3.42,16.1,8.1
226,H19-1555,Acute myeloid leukemia and related precursor n...,AML,Acute Myeloid Leukemia/Acute Lymphoblastic Leu...,70,"AML. Relapsed AML (+RUNX1, TP53, SRSF2, SETBP1...","1-3. Bone marrow, right posterior iliac crest;...",94.0,NaN,1.0,...,28.1,33.7,82,0.0,0.0,0,33,3.11,13.2,3.0
227,H19-1570,Acute myeloid leukemia and related precursor n...,AML,Acute Myeloid Leukemia/Acute Lymphoblastic Leu...,12,MDS. Evaluation of cellularity; also history o...,"1, 3. Bone marrow, right posterior iliac crest...",20.0,2.0,5.0,...,29.2,35.1,83,23.0,44.7,0,108,3.01,12.6,2.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1163,H20-6318,Mature B-cell neoplasm,Plasma cell myeloma,Plasma Cell Myeloma,79,Multiple Myeloma \nMGUS vs MM,"1-3. Bone marrow, right posterior iliac crest,...",1.0,1.0,9.0,...,32.1,33.3,96,13.2,53.0,0.0,263,3.58,13.6,7.4
1164,H20-748,Mature B-cell neoplasm,Plasma cell myeloma,Plasma Cell Myeloma,78,Multiple Myeloma. Plasmacytoma; rule out MM; i...,"1-3. Bone marrow, left posterior iliac crest, ...",1.0,1.0,9.0,...,33.8,33.8,100,5.0,64.4,0.0,309,3.20,11.7,7.6
1166,H20-934,Mature B-cell neoplasm,Plasma cell myeloma,Plasma Cell Myeloma,59,Multiple Myeloma\nPre-transplant on RVD,"1-3. Bone marrow, right posterior iliac crest,...",1.0,2.0,7.0,...,29.5,32.2,92,12.3,74.6,0.0,648,4.48,17.4,8.0
1167,H21-7213,Mature B-cell neoplasm,Plasma cell myeloma,Plasma Cell Myeloma,66,Multiple Myeloma,"1-3. Bone marrow, left posterior iliac crest, ...",NaN,NaN,NaN,...,31.0,32.9,94,9.0,44.0,0.0,184,4.06,15.9,4.9


In [11]:
case_df = case_df.drop(columns=['Pathology Report'])

In [12]:
case_df.to_csv('/Users/hasan/Library/Mobile Documents/com~apple~CloudDocs/Projects/Heme Report OpenAI/updated_case_list_with or without_cbc.csv', index=False)
case_df_cbc.to_csv('/Users/hasan/Library/Mobile Documents/com~apple~CloudDocs/Projects/Heme Report OpenAI/updated_case_list_with_cbc.csv', index=False)

### Will come back to subsequent cells once GPT labels cases as having characteristic lab abnormality

In [26]:
filtered_dfs = []

categories = case_df['General Dx'].unique()

for category in categories:
    category_df = case_df[case_df['General Dx'] == category]
    category_df = category_df.sort_values(by='year', ascending=False)
    n_rows = min(10, len(category_df))
    top_category_df = category_df.head(n_rows)
    filtered_dfs.append(top_category_df)


filtered_df = pd.concat(filtered_dfs)
filtered_df.reset_index(drop=True, inplace=True)

In [28]:
filtered_df.to_csv('final_case_list.csv', index=False)